In [2]:
!pip install xgboost
!pip install model_selection

     |████████████████████████████████| 157.5 MB 51 kB/s s eta 0:00:01    |██████████▎                     | 50.7 MB 68.9 MB/s eta 0:00:02     |██████████▊                     | 52.9 MB 68.9 MB/s eta 0:00:02     |███████████████▋                | 76.6 MB 26.7 MB/s eta 0:00:04     |██████████████████████          | 108.7 MB 26.7 MB/s eta 0:00:02     |██████████████████████████▋     | 130.8 MB 8.8 MB/s eta 0:00:04     |█████████████████████████████   | 142.7 MB 8.8 MB/s eta 0:00:02     |██████████████████████████████▉ | 152.0 MB 11.7 MB/s eta 0:00:01     |███████████████████████████████▋| 155.5 MB 11.7 MB/s eta 0:00:01


In [7]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.preprocessing import LabelEncoder
from google.cloud import storage, bigquery

In [14]:
client = bigquery.Client()
sql = "select * from `bigquery-public-data.ml_datasets.census_adult_income`"
df = client.query(sql).to_dataframe()

In [17]:
target = df["income_bracket"]
features = df.drop("income_bracket",1)
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)
print(target_encoded)

[0 1 1 ... 0 0 0]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)
dtrain = xgb.DMatrix(X_train._get_numeric_data(), label=y_train)
dtest = xgb.DMatrix(X_test._get_numeric_data(), label=y_test)

In [34]:
param = {
    'max_depth': 3,
    'eta': 0.3,
    'silent': 1,
    'objective': 'multi:softprob',
    'num_class': 3
}
num_round = 100
bst = xgb.train(param, dtrain , num_round)

[13:36:42] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:36:42] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [35]:
preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])
print(precision_score(y_test, best_preds, average='macro'))

0.8139098096663238


In [37]:
model = 'model.bst'
bst.save_model(model)
bucket = storage.Client().bucket('cld_ml')
blob = bucket.blob('{}/{}'.format('ml/models/', model))
blob.upload_from_filename(model)